# テキスト前処理

In [ ]:
# import csv
# import yaml
# import os
# import glob
# import sys
# sys.path.append('/home/takeshun256/PausePrediction/src/analyze-J-MAC')
# sys.path.append('/home/takeshun256/PausePrediction/src/vad_tool')
# from audiobook_yaml_parser import extract_yaml_data
# from py_webrtcvad_test import getVadSection
# from vad_tool import VAD_Segmenter
# from audiobook_dataset_builder import AudiobookDatasetBuilder
# from audiobook_script_extractor import AudiobookScriptExtractor
# # python==3.11以上の場合はimportしない
# if sys.version_info < (3, 11):
#     from text_preprocessing_preprocessor import AudiobookScriptPreprocessor
# # from text_preprocessing import AudiobookScriptPreprocessor
# from julius_lab_analysis import JuliusLabAnalyzer

# import numpy as np
# %matplotlib inline
# from matplotlib import pyplot as plt
# import japanize_matplotlib
# import webrtcvad
# from pprint import pprint
# import pandas as pd
# import seaborn as sns
# from tqdm import tqdm
# from typing import List, Tuple, Dict, Any, Union

# conda activate pyopenjtalk_julius

# set path
import sys

sys.path.append("/home/takeshun256/PausePrediction")

# import standard library
from pprint import pprint
from pathlib import Path
import yaml
import pandas as pd

# import own library
from config import DATA_DIR, DATA_TAKESHUN256_DIR, SRC_DIR, DATA_IN_ROOT_DIR
from src.analyze_jmac.text_preprocessing import (
    AudiobookScriptPreprocessor as Preprocessor,
)
from src.analyze_jmac.mecab import mecab_wakati_generator, mecab_detailed_generator

# define path
corpus_name = "jmac"
exp_name = "03_VAD_Adjusted"

exp_dir = Path(DATA_TAKESHUN256_DIR) / corpus_name / exp_name
yaml_file_path = Path(DATA_IN_ROOT_DIR) / corpus_name / "text_audio_dict_new.yaml"

assert exp_dir.exists()
assert yaml_file_path.exists()

In [ ]:
# load yaml
with open(yaml_file_path, "r") as f:
    yaml_data = yaml.safe_load(f)
pprint(yaml_data["audiobook_0"])

In [ ]:
# save original text

# txt save path
transcript_name = "transcript_original"

transcript_dict = {}
for audiobook_name, info in yaml_data.items():
    # audiobookの名称を修正
    # audiobook_1 -> AUDIOBOOK01
    audiobook_name_fixed = "AUDIOBOOK" + audiobook_name.split("_")[-1].zfill(2)
    transcript_lines = []
    for chapter_idx, chapter_info in enumerate(info["text"]):
        chapter_idx_str = str(chapter_idx).zfill(2)
        transcript_lines.append(
            f"{audiobook_name_fixed}_{chapter_idx_str}\t{chapter_info['sent']}"
        )
    transcript_dict[audiobook_name_fixed] = transcript_lines

pprint(transcript_lines[:3])

# save
# exp_dir / audiobook_name_fixed 下に {trascript_name}.txt として保存
# 1行1文: "{audiobook_name_fixed}_{chapter_idx_str} {chapter_info["sent"]}"

for audiobook_name_fixed, transcript_lines in transcript_dict.items():
    file_path = exp_dir / audiobook_name_fixed / f"{transcript_name}.txt"
    file_path.parent.mkdir(parents=True, exist_ok=True)
    with open(file_path, "w") as f:
        f.write("\n".join(transcript_lines))
    print(f"{file_path} saved.")

# for remove files
# shell command
# find /data2/takeshun256/03_VAD_Ajusted -name "transcript_original.txt"
# find /data2/takeshun256/03_VAD_Ajusted -name "transcript_original.txt" -exec rm {} \;

In [ ]:
# blacket removed and save kanji text

# 処理対象のファイル名
input_transcript_name = "transcript_original"

# txt save path
transcript_name = "transcript_kanji"

# exp_dirの下にある書くフォルダについて処理
input_transcript_dict = {}
for audiobook_dir in exp_dir.iterdir():
    input_file_path = audiobook_dir / f"{input_transcript_name}.txt"
    assert input_file_path.exists(), f"{input_file_path} does not exist."
    with open(input_file_path, "r") as f:
        lines = f.read().split("\n")
    input_transcript_dict[audiobook_dir.name] = lines

pprint(input_transcript_dict["AUDIOBOOK01"][:3])

# preprocess
transcript_dict = {}
lambda_func = (
    lambda line: line.split("\t")[0]
    + "\t"
    + Preprocessor.remove_brackets_to_kanji(line.split("\t")[1])
)
for audiobook_name, transcript_lines in input_transcript_dict.items():
    transcript_lines = [lambda_func(line) for line in transcript_lines]
    transcript_dict[audiobook_name] = transcript_lines

pprint(transcript_dict["AUDIOBOOK01"][:3])


# save
# exp_dir / audiobook_name_fixed 下に {trascript_name}.txt として保存
# 1行1文: "{audiobook_name_fixed}_{chapter_idx_str} {chapter_info["sent"]}"

for audiobook_name_fixed, transcript_lines in transcript_dict.items():
    file_path = exp_dir / audiobook_name_fixed / f"{transcript_name}.txt"
    file_path.parent.mkdir(parents=True, exist_ok=True)
    with open(file_path, "w") as f:
        f.write("\n".join(transcript_lines))
    print(f"{file_path} saved.")

In [ ]:
# blacket removed and save kanji text

# 処理対象のファイル名
input_transcript_name = "transcript_original"

# txt save path
transcript_name = "transcript_furigana"

# exp_dirの下にある書くフォルダについて処理
input_transcript_dict = {}
for audiobook_dir in exp_dir.iterdir():
    input_file_path = audiobook_dir / f"{input_transcript_name}.txt"
    assert input_file_path.exists(), f"{input_file_path} does not exist."
    with open(input_file_path, "r") as f:
        lines = f.read().split("\n")
    input_transcript_dict[audiobook_dir.name] = lines

pprint(input_transcript_dict["AUDIOBOOK01"][:3])

# preprocess
transcript_dict = {}
lambda_func = (
    lambda line: line.split("\t")[0]
    + "\t"
    + Preprocessor.remove_brackets_to_furigana(line.split("\t")[1])
)
for audiobook_name, transcript_lines in input_transcript_dict.items():
    transcript_lines = [lambda_func(line) for line in transcript_lines]
    transcript_dict[audiobook_name] = transcript_lines

pprint(transcript_dict["AUDIOBOOK01"][:3])


# save
# exp_dir / audiobook_name_fixed 下に {trascript_name}.txt として保存
# 1行1文: "{audiobook_name_fixed}_{chapter_idx_str} {chapter_info["sent"]}"

for audiobook_name_fixed, transcript_lines in transcript_dict.items():
    file_path = exp_dir / audiobook_name_fixed / f"{transcript_name}.txt"
    file_path.parent.mkdir(parents=True, exist_ok=True)
    with open(file_path, "w") as f:
        f.write("\n".join(transcript_lines))
    print(f"{file_path} saved.")

In [ ]:
# kanji text to wakati text

# 処理対象のファイル名
input_transcript_name = "transcript_kanji"

# txt save path
transcript_name = "transcript_wakati"

# exp_dirの下にある書くフォルダについて処理
input_transcript_dict = {}
for audiobook_dir in exp_dir.iterdir():
    input_file_path = audiobook_dir / f"{input_transcript_name}.txt"
    assert input_file_path.exists(), f"{input_file_path} does not exist."
    with open(input_file_path, "r") as f:
        lines = f.read().split("\n")
    input_transcript_dict[audiobook_dir.name] = lines

pprint(input_transcript_dict["AUDIOBOOK01"][:3])

# preprocess
transcript_dict = {}
lambda_func = (
    lambda line: line.split("\t")[0]
    + "\t"
    + " ".join(mecab_wakati_generator(line.split("\t")[1]))
)
for audiobook_name, transcript_lines in input_transcript_dict.items():
    transcript_lines = [lambda_func(line) for line in transcript_lines]
    transcript_dict[audiobook_name] = transcript_lines

pprint(transcript_dict["AUDIOBOOK01"][:3])


# save
# exp_dir / audiobook_name_fixed 下に {trascript_name}.txt として保存
# 1行1文: "{audiobook_name_fixed}_{chapter_idx_str} {chapter_info["sent"]}"

for audiobook_name_fixed, transcript_lines in transcript_dict.items():
    file_path = exp_dir / audiobook_name_fixed / f"{transcript_name}.txt"
    file_path.parent.mkdir(parents=True, exist_ok=True)
    with open(file_path, "w") as f:
        f.write("\n".join(transcript_lines))
    print(f"{file_path} saved.")

In [ ]:
# kanji text to morpheme texts

# 処理対象のファイル名
input_transcript_name = "transcript_kanji"

# txt save path
transcript_dir_name = "transcript_morpheme"

# exp_dirの下にある書くフォルダについて処理
input_transcript_dict = {}
for audiobook_dir in exp_dir.iterdir():
    input_file_path = audiobook_dir / f"{input_transcript_name}.txt"
    assert input_file_path.exists(), f"{input_file_path} does not exist."
    with open(input_file_path, "r") as f:
        lines = f.read().split("\n")
    input_transcript_dict[audiobook_dir.name] = lines

pprint(input_transcript_dict["AUDIOBOOK01"][:3])

# preprocess
transcript_dict = {}


def lambda_func(line):
    line = line.split("\t")
    detailed_list = list(mecab_detailed_generator(line[1]))  # namedtupleのリスト
    transcirpt_line = pd.DataFrame(detailed_list)
    return line[0], transcirpt_line


for audiobook_name, transcript_lines in input_transcript_dict.items():
    transcript_lines = [lambda_func(line) for line in transcript_lines]
    transcript_dict[audiobook_name] = transcript_lines

pprint(transcript_dict["AUDIOBOOK01"][:3])


# save
# exp_dir / audiobook_name_fixed/ transcript_dir_name 下に {trascript_line[0]}.txt として保存

for audiobook_name_fixed, transcript_lines in transcript_dict.items():
    transcript_dir = exp_dir / audiobook_name_fixed / transcript_dir_name
    transcript_dir.mkdir(parents=True, exist_ok=True)
    for transcript_line in transcript_lines:
        file_path = transcript_dir / f"{transcript_line[0]}.txt"
        transcript_line[1].to_csv(file_path, sep="\t", index=False, header=False)
    print(f"{transcript_dir} saved.")